In [4]:
# -*- coding: utf8 -*-
#########################IMPORTS############################################
from os.path import expanduser
import codecs, optparse
import re, random
import sys,os,time
import string
import yaml, YamlDuplicates
import ParFuMor as PFM
from ParFuMor import *
import pickle
#from cellbell import ding

def ding():
    os.system('afplay /System/Library/Sounds/Submarine.aiff')

### Javascript from :
https://stackoverflow.com/questions/12544056/how-to-i-get-the-current-ipython-notebook-name

In [5]:
%%javascript
var kernel = IPython.notebook.kernel;
var thename = window.document.getElementById("notebook_name").innerHTML;
var command = "theNotebook = " + "'"+thename+"'";
kernel.execute(command);

<IPython.core.display.Javascript object>

In [6]:
print(theNotebook)

Kalaba3-Lexique-Series


# Gestion partagée des numéros à traiter

le block suivant permet de partager les numéros à traiter entre les différents Notebooks.
- %store -r variable lit la variable dans le stock
- %store variable stocke la variable

In [7]:
%store -r numerosKalaba
# numerosKalaba=[4]
print numerosKalaba

[1]


In [8]:
devnag=False
devnag=True

kalabaVoyelles=u"ieaou"
kalabaConsonnes=u"ptkbdgmnNfTsSvDzZrljw"
kalabaIPA=[u"ʃ",u"ʒ",u"ŋ",u"θ", u"ð"]
fixedPhonInventory=True

In [9]:
home = expanduser("~")
repertoire=home+"/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/00-ProjetKalaba/"
serie=repertoire+"20-"
nomsKalabas=[serie+"K%d/"%num for num in numerosKalaba]
if devnag:
    declarationsExt="dn"
else:
    declarationsExt="tex"


In [10]:
def getExemple(lDict):
    exKey=lDict.keys()[0]
    if isinstance(lDict[exKey],dict):
        result=getExemple(lDict[exKey])
    elif isinstance(lDict[exKey],list):
        result=lDict[exKey][0]
    return result

In [11]:
nomDeclaration="Declarations.%s"%declarationsExt
nomDeclarationRad="Declarations-Radicaux.%s"%declarationsExt
nomDeclarationDec="Declarations-Decoupages.%s"%declarationsExt
nomTableauxRad="Tableaux-Gloses.yaml"
nomTableaux="Tableaux.yaml"

In [12]:
debug=0
print_glose=True
print_cloze=True
print_radicaux=True
cloze_expanded=True
numeros={'1':'Un','2':'Deux','3':'Trois','4':'Quatre','5':'Cinq'}
personnes={'1sg':'UnSg','2sg':'DeuxSg','3sg':'TroisSg',
           '1du':'UnDu','2du':'DeuxDu','3du':'TroisDu',
           '1pau':'UnPau','2pau':'DeuxPau','3pau':'TroisPau',
           '1pl':'UnPl','2pl':'DeuxPl','3pl':'TroisPl'}

if devnag:
    commandGrapho="\\newcommand{\\%s}{\\strutgb{\\graphoSkip}\\grapho{\\dn %s}}"    
    #commandGrapho="\\newcommand{\\%s}{\\strutgb{0pt}{\\dn %s}}"
else:
    commandGrapho="\\newcommand{\\%s}{\\strutgb{\\graphoSkip}\\grapho{%s}}"    
    #commandGrapho="\\newcommand{\\%s}{\\strutgb{0pt}{%s}}"

commandPhono=u"\\newcommand{\\%sP}{\\textipa{%s}}"
commandPhonoDecoupe=u"\\newcommand{\\%sD}{\\textipa{%s}}"
commandGlose=u"\\newcommand{\\%sG}{\\textGlose{%s}}"

In [13]:
def parse_grapho(graphie):
    def change_monograms(chaine):
        result=""
        for monogramme in chaine:
            if monogramme in monogrammes:
                result+=monogrammes[monogramme]
            else:
                result+=monogramme
        return result

    def change_bigrams(chaine):
        if monogrammes:
            result=change_monograms(chaine)
        else:
            result=chaine
        if bigrammes:
            regexBigrammes="|".join(bigrammes.keys())
            changements=re.findall(regexBigrammes,result)
            for bigramme in changements:
                result=result.replace(bigramme,bigrammes[bigramme])
        return result

    ######
    #
    # Changer la regexp pour s'adapter à phonology.yaml
    #
    ######
    
    chunks=re.findall(ur"\s+|[ptkqbdgmnNfFTsSvVDzZjwRrlLyvWJ]?[aeiouAEIUO]?", graphie,re.U)
    result=[]
    if debug: print graphie
    for chunk in chunks:
        if debug: print chunk,
        #ATTENTION : YAML interprète "no" comme FALSE
        if chunk in syllabes.keys():
            result.append(syllabes[chunk])
        else:
            result.append(chunk)
    if debug: print
    return change_bigrams("".join(result))

In [14]:
def parse_cloze(glose):
    if debug: print [glose]
    chunks=re.findall(r"\\cacherGloses{([^}]*)?}|(\w+)", glose,re.UNICODE)
    result=[]
    for chunk in chunks:
        result.extend([x for x in chunk if x!=""])
    return (u"%s;"%len(result)+";".join(result),"".join(result))

In [15]:
def recoder(chaine,table,boolUTF8=True):
    if type(chaine)==str:
        temp=unicode(chaine.decode('utf8')).translate(table)
        result=temp
    elif type(chaine)==unicode:
        result=chaine.translate(table)
    if boolUTF8:
        return result
    else:
        return result.encode("utf8")
#translit=string.maketrans(u'iueoaftgzZvjkSpN',u'tgazpHTGZJVkXyxI')

In [16]:
def escapeGabarit(forme):
    result=forme
    for i in "123":
        result=result.replace(i,"\\textnormal{C}$_{%s}$"%i)
        result=result.replace(str(int(i)+3),"\\textnormal{C}$_{%s}^{m}$"%i)
    result=result.replace("V","\\textnormal{V}")        
    result=result.replace("A","\\textnormal{V}$_{\\textnormal{\\tiny apo}}$")        
    return result

def escapeRacine(forme):
    m=re.match(ur"(.*)racine\(([^)]+)\)x(.*)",forme)
    if m:
        prefixe=m.group(1)
        if debug: print prefixe
        racine="\\racine{%s}"%m.group(2)
        if debug: print racine
        suffixe=m.group(3)
        suffixes=re.split(ur"([-+])",suffixe)
        gabarit="$\\times$\\gabarit{%s}"%escapeGabarit(suffixes[0])
        terminaison="".join(suffixes[1:])
        return prefixe+racine+gabarit+terminaison
    else:
        return forme



In [17]:
def boolPhonInventory(forme,consonnes,voyelles):
    lForme=forme
    for c in kalabaIPA:
        lForme=lForme.replace(c,"")
    return re.findall(ur"[^%s%s]"%(consonnes,voyelles),lForme)
    
def boolHiatus(forme,voyelles):
    return re.findall(ur"(?=([%s][%s]))"%(voyelles,voyelles),forme)
    
def boolVoyelleInitiale(forme,voyelles):
    if forme[0] in voyelles:
        return True
    else:
        return False
    
def bilanVoyellesInitiales(voc,cons,label,cat,thresh=0.1):
    nbVoc=len(voc)
    nbCons=len(cons)
    nbTot=nbVoc+nbCons
    ratio=float(nbVoc)/nbTot
    if ratio>=thresh and cat not in "PREP DET".split():
        print "\t",cat,label,"Voyelles initiales",float(nbVoc)/nbTot
        ding()
    return

def pourcent(nombre):
    return u"%2.0f%%"%(100*nombre)

def bilanPhon(cat,thresh=0.1):
    nbVocStems=len(vocRad)
    nbConsStems=len(consRad)
    nbStems=nbVocStems+nbConsStems
    ratioVocStems=float(nbVocStems)/nbStems
    nbVocForms=len(vocForm)
    nbConsForms=len(consForm)
    nbForms=nbVocForms+nbConsForms
    ratioVocForms=float(nbVocForms)/nbForms
    nbHiatus=len(hiatusForm)
    ratioHiatus=float(nbHiatus)/nbForms
    nbHorsInv=len(horsPhon)
    if ratioVocForms>=thresh and cat not in "PREP DET".split():
        print "\t",cat,"Formes","Voyelles initiales",pourcent(ratioVocForms),nbForms
        ding()
    elif ratioVocStems>=thresh and cat not in "PREP DET".split():
        print "\t",cat,"Radicaux","Voyelles initiales",pourcent(ratioVocStems),nbStems
        ding()
    if ratioHiatus>=thresh and cat not in "PREP DET".split():
        print "\t",cat,"hiatus",pourcent(ratioHiatus),nbForms
    if nbHorsInv>0:
        print "\t",", ".join(horsPhon)
    return
    
        

In [18]:
boolPhonInventory(u"teiuoit",kalabaConsonnes,kalabaVoyelles)

[]

In [19]:
for serie in nomsKalabas:
    print "Génération lexique :",serie.split("/")[-2], time.strftime("%H:%M:%S")


    '''
    Lecture des fichiers de paramètres
     1) Phonology.yaml pour 
         - les voyelles, les consonnes
         - la correspondance pour l'écrit (symbole => lettre, syllabe => lettres)
         - la correspondance pour l'oral
         - la désaccentuation et les déligatures du français
         - les gabarits ?
         - l'apophonie, les mutations consonantiques
     2) Stems.yaml pour
         - les catégories et les classes
         - les radicaux et les formes correspondantes
             - la première forme sert de forme de citation
     3) MorphoSyntax.yaml pour
         - définition de la valeur des formes verbales du français
         - définition des attributs de chaque catégorie
         - valeur par défaut pour chaque attribut
         - définition des cas pour SUJ, OBJ, IND et PREP
         - .../...
         - définition des constituants et de l'ordre dans les syntagmes
         - définition des contractions du français (du => de le, etc.)
    '''
    with open(serie+"Phonology.yaml", 'r') as stream:
        phonology=yaml.safe_load(stream)
    with open(serie+"Stems.yaml", 'r') as stream:
        stems=yaml.safe_load(stream)
    lexiqueTestPrep=getExemple(stems["PREP"])
    lexiqueTestNoun=getExemple(stems["NOM"])
    lexiqueTestHyper=getExemple(stems["ADJ"])
    lexiqueTestVerbe=getExemple(stems["VER"])
    
    with open(serie+"MorphoSyntax.yaml", 'r') as stream:
        morphosyntax=yaml.safe_load(stream)
        
    '''
    Récupération des données du ParFuMor
    '''
    with open(serie+"Hierarchie-S2.pkl", 'rb') as input:
       PFM.hierarchieCF = pickle.load(input)
    with open(serie+"Lexique-S2.pkl", 'rb') as input:
       PFM.lexique = pickle.load(input)
    with open(serie+"Regles-S2.pkl", 'rb') as input:
       PFM.regles = pickle.load(input)

    if debug: print "Test PREP"; PFM.lexique.lexemes[lexiqueTestPrep]

    if fixedPhonInventory:
        consonnes=kalabaConsonnes
        voyelles=kalabaVoyelles
    else:
        consonnes=phonology["consonnes"]
        voyelles=phonology["voyelles"]
    gabarits=phonology["gabarits"]
    derives=phonology["derives"]
    nom_classe=phonology["nom_classe"]
    nom_apo=phonology["apophonies"]
    nom_mut=phonology["mutations"]
    syllabes=phonology["syllabes"]

    if "Cas" in morphosyntax:
        casSyntagmes=morphosyntax["Cas"]
    else:
        casSyntagmes=""
    lexiquePrepositions=[stems["PREP"][x][0] for x in stems["PREP"]]
    casPreposition={}
    for preposition in lexiquePrepositions:
        prep=preposition.upper()
        if casSyntagmes and prep in casSyntagmes:
            casPreposition[preposition]=casSyntagmes[prep]
        elif casSyntagmes and "PREP" in casSyntagmes:
            casPreposition[preposition]=casSyntagmes["PREP"]
        else:
            casPreposition[preposition]=""
    glosePREP={x:u"%s%s"%(x.upper(),cacherGloses("["+casPreposition[x].strip("+").capitalize()+"]")) for x in casPreposition if "+" in casPreposition[x]}

    try:
        __IPYTHON__ 
        ipython=True
    except: 
        ipython=False

    
    '''
    Constitution d'un entête pour les fichiers TEX en sortie
        - nom du script
        - date du run
    '''
    texHeader=[]
    version=theNotebook
    texHeader.append("%% script : "+version)
    texHeader.append('%%%% run : %s' % time.strftime("%y%m%d-%H%M"))
    if devnag:
        texHeader.append('@hindi')
    if debug: print "\n".join(texHeader)
    
    if ipython or True:
    #    lexeme_nom="lexemes.txt"
    #    phrase_nom="phrases.txt"
        pass
    else:
        parser=optparse.OptionParser()
        parser.add_option("-o", "--out", dest="outfile", action="store_true", help="write to FILE")
        parser.add_option("-c", "--cloze", dest="print_cloze", action="store_true", help="write a CLOZE FILE")
        parser.add_option("-l", "--lexicon", dest="print_lexique", action="store_true", help="append a lexicon")
        parser.add_option("-r", "--roots", dest="print_racines", action="store_true", help="append a root list")

        (options, args) = parser.parse_args()
        lexeme_nom=args[0]
        phrase_nom=args[1]

    phonoIn =  unicode(phonology["translations"]["grapho"]["in"])
    graphoIn = [ord(char) for char in phonoIn]
    graphoOut = unicode(phonology["translations"]["grapho"]["out"])
    translit = dict(zip(graphoIn, graphoOut))
    if "monogrammes" in phonology["translations"]:
        monogrammes = phonology["translations"]["monogrammes"]
    else:
        monogrammes = {}
    bigrammes = phonology["translations"]["bigrammes"]

    accentedIn = unicode(phonology["translations"]["deaccent"]["in"])
    deaccentIn = [ord(char) for char in accentedIn]
    deaccentOut = unicode(phonology["translations"]["deaccent"]["out"])
    deaccent = dict(zip(deaccentIn, deaccentOut))

    deligatures=phonology["translations"]["deligatures"]

    tipaIn = unicode(phonology["translations"]["ipa"]["in"])
    ipaIn = [ord(char) for char in tipaIn]
    ipaOut = unicode(phonology["translations"]["ipa"]["out"])
    toipa = dict(zip(ipaIn, ipaOut))

    tableaux={}
    tableauxGloses={}
    gloseClozes={}
    declarations=[]
    declarationsRad=[]
    declarationsDec=[]
    vocRatio={}
    hiatusRatio={}
    
    '''
    Pour chaque catégorie
    '''
    for categorie in PFM.lexique.catLexeme:
        '''
        compter le nombre de radicaux et de formes qui commencent par une voyelle et par une consonne
        compter le nombre de hiatus dans les formes
        '''
        vocRad=[]
        consRad=[]
        vocForm=[]
        consForm=[]
        hiatusForm=[]
        hiatusCounts={}
        horsPhon=[]
        '''
        si la catégorie n'est pas dans le tableau, il faut l'ajouter dans les trois structures :
            tableaux : les lignes à ajouter à la fin du document LaTeX sans les gloses
            tableauxGloses : les lignes à ajouter à la fin du document LaTeX avec les gloses
            gloseClozes : les informations sur les mots pour les questions CLOZE 
        '''
        if not categorie in tableaux:
            tableaux[categorie]=set()
            tableauxGloses[categorie]=set()
            gloseClozes[categorie]=[]
        if debug: print categorie
        '''
        pour chaque lexème de la catégorie
        '''
        for lexeme in PFM.lexique.catLexeme[categorie]:
            stemLexeme=PFM.lexique.lexemes[lexeme].stem
            if boolVoyelleInitiale(stemLexeme,voyelles):
                vocRad.append(stemLexeme)
            else:
                consRad.append(stemLexeme)
            if debug: print "lexeme,PFM.lexique.lexemes[lexeme]",lexeme,PFM.lexique.lexemes[lexeme].stem
            '''
            pour chaque case du paradigme du lexème
            '''
            for case in PFM.lexique.lexemes[lexeme].paradigme.cases:
                formLexeme=case.forme
                if boolVoyelleInitiale(formLexeme,voyelles):
                    vocForm.append(formLexeme)
                else:
                    consForm.append(formLexeme)
                if boolHiatus(formLexeme,voyelles):
                    hiatusForm.append(formLexeme)
                if boolPhonInventory(formLexeme,consonnes,voyelles):
                    horsPhon.append(formLexeme)
                if debug: print "case", case
                caseGlose=case.glose
                '''
                Les éléments des catégories majeures (N,V,Adj) sont représentés en minuscules
                    => les noms propres commencent par une capitale
                Les éléments grammaticaux sont représentés en majuscules
                '''
                if categorie in PFM.categoriesMajeures:
                    nom=PFM.lexique.lexemes[lexeme].nom
                else:
                    nom=PFM.lexique.lexemes[lexeme].nom.upper()
                    gloseCase=case.glose
                    if categorie=="PREP" and gloseCase in glosePREP:
                        caseGlose=glosePREP[gloseCase]
                    elif categorie=="PREP":
                        caseGlose=caseGlose.upper()
                ref=nom.replace("-","")
                ref=PFM.modifierGlose(ref,case.sigma,"ref")
                ref=recoder(ref,deaccent)
                
                
                '''
                Dealing with LaTeX names format for commands
                
                 1) no ligatures
                 2) no numbers
                '''
                
                for ligature in deligatures:
                    ref=ref.replace(ligature,deligatures[ligature])
                for pers in personnes:
                    ref=ref.replace(pers,personnes[pers])
                for num in numeros:
                    ref=ref.replace(num,numeros[num])
                
                    
                grapho=chaine2utf8(re.sub(ur"\s+",ur"",parse_grapho(recoder(case.forme,translit))))
                phonoRad=case.detoure.replace(" ","")
                phonoDec=escapeRacine(case.decoupe)
                if debug: print "phono Rad", phonoRad
                phono=case.forme.replace(" ","")

                '''
                Creating the DECLARATIONS TEX contents
                '''
                
                declarations.append(commandGrapho%(ref,grapho))
                declarations.append(commandPhono%(ref,phono))
                declarations.append(commandGlose%(ref,caseGlose))
                declarationsRad.append(commandGrapho%(ref,grapho))
                declarationsRad.append(commandPhono%(ref,phonoRad))
                declarationsRad.append(commandPhonoDecoupe%(ref,phonoDec))
                declarationsRad.append(commandGlose%(ref,caseGlose))
                declarationsDec.append(commandGrapho%(ref,grapho))
                declarationsDec.append(commandPhono%(ref,phonoDec))
                declarationsDec.append(commandGlose%(ref,caseGlose))
                
                '''
                Creating the TABLEAUX TEX contents
                '''
                
                tableauxGloses[categorie].add("\\"+ref+" & \\"+ref+"P & \\"+ref+"G \\\\")
                tableaux[categorie].add("\\"+ref+" & \\"+ref+"P & \\blanc{\\"+ref+"G} \\\\")
                
                '''
                Creating the CLOZE CSV contents
                '''
                
                vedette=nom.split(".")[0]
                gloses,strGlose=parse_cloze(case.glose)
#                print gloses,case.decoupe,case.detoure
                phono=case.forme.replace(" ","")
                if debug: print "grapho, recoder(phono,toipa)", [grapho, recoder(phono,toipa)]
                if cloze_expanded:
                    try:
                        if debug: print ref
                        cloze=u";".join([ref,vedette,
                                         categorie,
                                         recoder(case.radical,toipa),recoder(case.racine,toipa),
                                         recoder(phono,toipa),recoder(case.decoupe,toipa),
                                         case.sigma,
                                         strGlose,
                                         gloses])
                    except NameError:
                        print ref
                        cloze=u";".join([ref,vedette,
                                         categorie,
                                         recoder(case.radical,toipa),recoder(case.racine,toipa),
                                         recoder(phono,toipa),
                                         strGlose,
                                         gloses])
                else:
                    try:
                        print ref
                        cloze=u";".join([ref,vedette,categorie,recoder(phono,toipa),recoder(case.decoupe,toipa),case.sigma,gloses])
                    except NameError:
                        print ref
                        cloze=u";".join([ref,vedette,categorie,recoder(phono,toipa),gloses])
                gloseClozes[categorie].append(cloze)
#         bilanVoyellesInitiales(vocRad,consRad,"Radicaux",categorie)
#         bilanVoyellesInitiales(vocForm,consForm,"Formes",categorie)
        bilanPhon(categorie)
        '''
        Écriture des fichiers en sortie
         1) déclarations sans les radicaux
         2) déclarations avec les radicaux
         3) données pour les questions CLOZE
         4) tableaux de vocabulaire avec gloses
         5) tableaux de vocabulaire sans gloses
        '''
        with codecs.open(serie+nomDeclaration, 'wb', encoding='utf8') as output:
            for declaration in texHeader+declarations:
        #        print type(declaration),declaration
                output.write(declaration+"\n")

        with codecs.open(serie+nomDeclarationRad, 'wb', encoding='utf8') as output:
            for declaration in texHeader+declarationsRad:
        #        print type(declaration),declaration
                output.write(declaration+"\n")

#         with codecs.open(serie+nomDeclarationDec, 'wb', encoding='utf8') as output:
#             for declaration in texHeader+declarationsDec:
#                 print type(declaration),declaration
#                 output.write(declaration+"\n")

        with codecs.open(serie+"Clozes.txt", 'wb', encoding='utf8') as output:
            for categorie in gloseClozes:
                output.write("#\t"+categorie+"\n#\n#\n")
                for cloze in gloseClozes[categorie]:
                    output.write(cloze+"\n")
                output.write("#\n#\n#\n")

        yaml.safe_dump(tableauxGloses, file(serie+nomTableauxRad, 'w'), encoding='utf-8', allow_unicode=True)
        yaml.safe_dump(tableaux, file(serie+nomTableaux, 'w'), encoding='utf-8', allow_unicode=True)
        if devnag:
            os.system('devnag %s'%(serie+nomDeclaration))
            os.system('devnag %s'%(serie+nomDeclarationRad))
            os.system('devnag %s'%(serie+nomDeclarationDec))    
print u"Génération terminée :", time.strftime("%H:%M:%S")

Génération lexique : 20-K1 05:10:29
Génération terminée : 05:10:32


In [20]:
ding()